# Lab 08
## Pavel Oreshin
### <a href='https://github.com/TermiNutZ/advstat-labs'> Link to repo </a>

# Assignment 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [12]:
ds_df = pd.read_csv("ds-boot-2.csv", index_col=0, sep='\t')

ds_df.shape

(40, 33)

In [13]:
ds_df.head()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
id,,,,,,,,,,,,,,,,,,,,,
11,6.48148,3.0,5.0,7.75000,0.0,7.16667,8.16667,9.66667,6.16667,9.0,...,1.66667,3.16667,0.0,0.00000,0.0,1.0,5.00,7.44,1.18,4.38
12,5.74074,4.0,8.0,7.33333,8.0,8.83333,9.75000,9.66667,9.00000,10.0,...,2.50000,5.50000,5.0,8.66667,8.0,4.5,4.25,8.93,2.00,6.03
25,7.59259,7.0,8.0,7.66667,8.0,9.66667,9.50000,6.16667,9.66667,6.5,...,3.50000,3.50000,9.0,6.50000,7.0,7.5,11.00,8.97,2.00,9.12
31,5.96297,4.0,8.0,9.33333,10.0,9.33333,7.00000,8.50000,9.66667,6.5,...,0.00000,0.00000,0.0,0.00000,0.0,4.0,6.25,8.93,1.82,6.41
48,5.44444,1.0,3.5,6.41667,9.0,8.50000,7.08333,6.33333,9.00000,7.0,...,0.00000,0.00000,0.0,0.00000,0.0,0.0,3.00,8.08,1.36,3.67


### define boostrap procedure

In [39]:
def bootstrap(x, B=1000, stat=np.mean):
    n = x.shape[0]
    xb = np.random.choice(x, (B, n), replace=True)
    
    xb_stat = stat(xb, axis=1)
    xb_stat.sort()

    ci = np.percentile(xb_stat, [2.5, 97.5])
    
    sd = np.std(xb_stat)
    
    return np.mean(xb_stat), sd

### calculate for mean and median

In [40]:
for stat_func in [np.mean, np.median]:
    print(stat_func)
    for i in range(1, 6):
        print("y%s : "%i, bootstrap(ds_df['y%s' % i], 1000, stat_func))
    print()

<function mean at 0x7f11b00d0048>
y1 :  (4.0041125000000006, 0.43816142127502511)
y2 :  (5.514637500000001, 0.56014148031434885)
y3 :  (7.3102907500000001, 0.47757277039937845)
y4 :  (1.53561525, 0.10840471924661536)
y5 :  (5.6312530000000001, 0.44098504395387378)

<function median at 0x7f118a9a3d08>
y1 :  (4.00875, 0.56390241842006672)
y2 :  (6.2220000000000004, 0.65897439252219814)
y3 :  (8.3061799999999995, 0.2738750766316646)
y4 :  (1.8636349999999997, 0.13081537285426359)
y5 :  (6.17835, 0.41144553406252937)



## Linear regression

In [31]:
df_features = ds_df.drop(['y%s' % i for i in range(1, 6)], axis=1)

In [158]:
def bootstrap_regression(x, y, B=1000):
    n = x.shape[0]
    bootstrap_indexes = np.random.choice(range(n), (B, n), replace=True)
    
    regressor = LinearRegression()
    coeffs = []
    for row in bootstrap_indexes:
        X_train = x.iloc[row]
        y = y.iloc[row]
        regressor.fit(X_train, y)
        coeffs.append(regressor.coef_)

    means = np.mean(coeffs, axis=0)
    ci = np.percentile(coeffs, [2.5, 97.5], axis=0)
    
    ci = ci.T
    return means, ci
    

### Will take only part of coefficients cause of overfitting

In [174]:
means, ci = bootstrap_regression(df_features[['p%s' % i for i in range(1,25)]], ds_df['y1'])

In [175]:
for i in range(len(means)):
    print("c%s" % i, means[i], ci[i])

c0 -0.00562479858703 [-0.35562378  0.56418495]
c1 -0.0213824078245 [-0.27168889  0.30953548]
c2 -0.0035001080899 [-0.54346096  0.47804814]
c3 0.0149771809725 [-0.42321221  0.5558626 ]
c4 -0.00285152471002 [-0.46730213  0.32042658]
c5 0.0566148835726 [-0.6444864  0.4561134]
c6 -0.00652427609573 [-0.56768302  0.48075963]
c7 0.000558288655654 [-0.4369267   0.38427672]
c8 0.00155671441457 [-0.62311336  0.4902558 ]
c9 -0.00366975811663 [-0.46733822  0.36755952]
c10 -0.00732766098002 [-0.31879944  0.6225743 ]
c11 -0.0387270588403 [-0.59881638  0.57318769]
c12 0.0220154132789 [-0.41502146  0.42539631]
c13 -0.0218728737989 [-0.28992526  0.3490447 ]
c14 -0.015118034518 [-0.27381191  0.25049124]
c15 0.0179706392263 [-0.21665513  0.23384877]
c16 -0.0116212472044 [-0.21412507  0.20234546]
c17 0.0229409790862 [-0.24652819  0.29835823]
c18 -0.0330563117079 [-0.25476036  0.27400763]
c19 0.00898783564605 [-0.30113769  0.34061811]
c20 0.0315333739592 [-0.24493732  0.31723801]
c21 -0.0268609054663 [-0.2

# Assignment 2

In [87]:
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from tqdm import tqdm

### Goel-Okumoto model

In [88]:
def s_shaped(a, b):
    def distrib(t):
        return a*(1-np.e**(-b*t))
    return distrib

In [89]:
ds_df = pd.read_csv("ds-boot-1.csv", index_col=0, sep='\t')

### Algorithm for bootstrap: Take half of days and add to found bugs integer from [-2, 2]. <br/>
### Then use optimization on mean squared error for our function

In [112]:
def least_sq_err(vals):
    a, b = vals
    func = s_shaped(a, b)
    predictions = [func(t) for t in np.arange(len(current_cumsum))]
    return mean_squared_error(current_cumsum, predictions)


#constraints for minimization
bnds = ((0, None), (0, 1))


B = 1000
results_a = []
results_b = []
change_ind_size = int(0.5 * ds_df.shape[0])
for i in tqdm(range(B)):
    bugs = ds_df['bugsPerDay'].copy()
    
    change_indexes = np.random.choice(np.arange(len(bugs)), size=change_ind_size)
    
    for ind in change_indexes:
        bugs.iloc[ind] += np.random.randint(-2, 3)
        bugs.iloc[ind] = max(0, bugs.iloc[ind])
        
    current_cumsum = np.cumsum(bugs)
    
    result = minimize(least_sq_err, [6000., 0.001], tol=1e-25, bounds=bnds)
    a, b = result.x
    
    results_a.append(a)
    results_b.append(b)
    
print("a: {} +- 1.96 * {}".format(np.mean(results_a), np.std(results_a)))
print("b: ({} +- 1.96 * {})".format(np.mean(results_b), np.std(results_b)))


  0%|          | 2/1000 [00:01<14:15,  1.17it/s]
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/home/pavel/anaconda2/envs/Python36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/pavel/anaconda2/envs/Python36/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/pavel/anaconda2/envs/Python36/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 1000/1000 [10:10<00:00,  1.64it/s]

a: 14225.724322624412 +- 1.96 * 2535.4618072156272
b: (0.0004103678875097642 +- 1.96 * 8.823647879662618e-05)
